In [50]:
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import StandardScaler
from train_utils import processTestData, columnsToDrop

In [6]:
!pwd

/Users/vrajagopalan/Work/GitHub/edge2ai_pred_maint-1/src


In [61]:
file = 'FD001'
Cycle_Alert_Threshold = 40

file_relative_path = "../data/"  ## To be changed to an environment variable

filename     = "test_" + file + ".csv"

test_df = pd.read_csv(file_relative_path + filename)
model_name = "pred_RUL.pkl"
model_relative_path = "../model/"
with open(model_relative_path + model_name, 'rb') as f:
    serialized_list = pickle.load(f)
scaler = serialized_list[0]
model_reg = serialized_list[1]
model_clf = serialized_list[2]
colsDrop = columnsToDrop(test_df)
X_test =  test_df.drop(columns= colsDrop).values
X_test = scaler.transform(X_test)
regr_results = model_reg.predict(X_test)
clf_results = model_clf.predict(X_test)
clf_probability =  model_clf.predict_proba(X_test)
#X_test_results = np.concatenate((X_test, regr_results, clf_results, clf_probability), axis=1) -- gives shape mismatches
#print(X_test.shape, regr_results.reshape((len(X_test),1)).shape, clf_probability[:,0].reshape((len(X_test),1)).shape, clf_results.reshape((len(X_test),1)).shape)
#change from (100, ) to (100, 1) shape to facilitate concatenation
regr_results = regr_results.reshape((len(X_test),1))
#here we take just the probability of failure and discard the probabilty of non failure i.e the 2nd value 
clf_probability =  clf_probability[:,0].reshape((len(X_test),1))
clf_results = clf_results.reshape((len(X_test),1))
#X_test_results = np.concatenate((X_test, regr_results, clf_results, clf_probability), axis=1)
#print(X_test.shape, X_test_results.shape) # we added 3 columns, RUL Regressor/ Forecaster, Failure or Non Failure based on threshold at training, Probability of Failure
X_results = np.concatenate(( regr_results, clf_results, clf_probability), axis=1)
column_values = ['RUL_FORECAST', 'FAIL_PREDICTION', 'FAIL_PROBABILITY']
results_df = pd.DataFrame(data=X_results, columns=column_values)
test_with_results_df = pd.concat([test_df, results_df], axis=1)
test_with_results_df.rename(columns = {'ENGINE_NUMBER': 'MACHINE_NUMBER'}, inplace=True)


test_with_results_df.head()
test_with_results_df.to_csv("../results/predictions_" + file+".csv")

In [60]:
results_df.head()

,RUL_FORECAST,FAIL_PREDICTION,FAIL_PROBABILITY
0,126.200455,0.0,0.879432
1,124.112259,0.0,0.879432
2,121.422997,0.0,0.879432
3,118.583107,0.0,0.879432
4,131.348526,0.0,0.879432
